## Evaluation exercise for Google Summer of Code / Energy cost of scientific software 2023

In this exercise we will look at the profiling of a package I have written in Python, that performs data preprocessing and principal component analysis of monsoon data. The profiling is done with the help of a open-source tool, Scalene, a high-precision CPU and memory profiler. 

In [2]:
from mssa import preprocessing as pre
from mssa import mssa

# Load Scalene
%load_ext scalene

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version.


**Profile of the data preprocessing:**

In [8]:
%scrun pre.read_data("test-data/TRMM-GPM_pr_Indian_region_1998.nc")

In [10]:
%scrun rainfall_data = pre.moving_mean(pre.stack(pre.read_data("test-data/TRMM-GPM_pr_Indian_region_1998.nc")), 60, 'time')

In [14]:
!scalene --cli --reduced-profile mssa/preprocessing.py

            Memory usage: ■■■▀▀ (max: 30.196 MB, growth rate: 101%)            
      mssa/preprocessing.py: % of time = 100.00% (1.776s) out of 1.776s.       
       ╷       ╷       ╷       ╷        ╷       ╷               ╷       ╷      
       │Time   │–––––– │–––––– │Memory  │–––––– │–––––––––––    │Copy   │      
  Line │Python │native │system │Python  │peak   │timeline/%     │(MB/s) │mss…  
╺━━━━━━┿━━━━━━━┿━━━━━━━┿━━━━━━━┿━━━━━━━━┿━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━┿━━━━━╸
     1 │    9% │    6% │   2%  │ 100%   │   10M │▄  33%         │       │impo  
     2 │   54% │   23% │   6%  │  98%   │   20M │■▀  67%        │   108 │impo  
   ... │       │       │       │        │       │               │       │      
       ╵       ╵       ╵       ╵        ╵       ╵               ╵       ╵      
Top AVERAGE memory consumption, by line:
(1)     1:    10 MB                                                             
Top PEAK memory consumption, by line:
(1)     2:    20 MB                     

**Profile of the principal component analysis:**

In [13]:
%scrun mssa.pca(rainfall_data.dropna('time'))

In [15]:
!scalene --cli --reduced-profile mssa/mssa.py

            Memory usage: ▄■■▀▀ (max: 40.101 MB, growth rate: 100%)            
           mssa/mssa.py: % of time = 100.00% (5.651s) out of 5.651s.           
       ╷       ╷       ╷       ╷        ╷       ╷               ╷       ╷      
       │Time   │–––––– │–––––– │Memory  │–––––– │–––––––––––    │Copy   │      
  Line │Python │native │system │Python  │peak   │timeline/%     │(MB/s) │mss…  
╺━━━━━━┿━━━━━━━┿━━━━━━━┿━━━━━━━┿━━━━━━━━┿━━━━━━━┿━━━━━━━━━━━━━━━┿━━━━━━━┿━━━━━╸
     1 │   60% │   26% │  14%  │ 100%   │   40M │▄■▀▀ 100%      │   128 │from  
   ... │       │       │       │        │       │               │       │      
       ╵       ╵       ╵       ╵        ╵       ╵               ╵       ╵      
Top PEAK memory consumption, by line:
(1)     1:    40 MB                                                             
